In [2]:
# !python -m pip install corextopic
# !python -m pip install networkx

In [31]:
import matplotlib.pyplot as plt
import pandas as pd 
import re
from sklearn.feature_extraction.text import CountVectorizer
from corextopic import corextopic as ct
from corextopic import vis_topic as vt # jupyter notebooks will complain matplotlib is being loaded twice
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
import scipy.sparse as ss
%matplotlib inline

In [90]:
newline = []

textstring = ''
with open('Hansard_preCovid.csv', 'r', encoding='utf-16-le') as inp, open('newtextfile.txt', 'w') as out:
    for line in inp:
      newline.append(re.findall(r'"(.*)"', line))
      
temp = pd.Series(newline)

initialDF_preCovid = pd.DataFrame({'Text': temp})
clearWords = stopwords.words('english')
new_Stopwords = ['in', 'the', 'said', 'like','must', 'many', 'also']
clearWords.extend(new_Stopwords)
lemma= WordNetLemmatizer()
initialDF_preCovid['Text'] = initialDF_preCovid['Text'].astype(str).str.lower()
initialDF_preCovid['Text'] = initialDF_preCovid['Text'].str.replace("|".join([r"^.*?speaker,", r"^.*?chair," ]), ' ', regex=True)
initialDF_preCovid['text_tokens'] = initialDF_preCovid['Text'].apply(word_tokenize)
initialDF_preCovid['text_tokens'] = initialDF_preCovid['text_tokens'].apply(lambda x: [item for item in x if item not in clearWords])
initialDF_preCovid['text_tokens'] = initialDF_preCovid['text_tokens'].apply(lambda x: [lemma.lemmatize(item) for item in x])
# do we love me or what? isn't this join thing amazing?
initialDF_preCovid['text_tokens'] = initialDF_preCovid['text_tokens'].apply(lambda x:' '.join([item for item in x if len(item)>3]))

The topic model assumes input is in the form of a doc-word matrix, where rows are documents and columns are binary counts. We'll vectorize our data, take the top 20,000 words, and convert it to a sparse matrix to save on memory usage. Note, we use binary count vectors as input to the CorEx topic model.

In [91]:
vectorizer = CountVectorizer(stop_words='english', max_features=20000, binary=True)
doc_word = vectorizer.fit_transform(initialDF_preCovid['text_tokens'].astype(str))
doc_word = ss.csr_matrix(doc_word)
doc_word.shape # n_docs x m_words


(305, 7778)

In [92]:
import numpy as np
# Get words that label the columns (needed to extract readable topics and make anchoring easier)
words = list(np.asarray(vectorizer.get_feature_names_out()))
not_digit_inds = [ind for ind,word in enumerate(words) if not word.isdigit()]
doc_word = doc_word[:,not_digit_inds]
words    = [word for ind,word in enumerate(words) if not word.isdigit()]

doc_word.shape # n_docs x m_words


(305, 7396)

In [43]:
# Train the CorEx topic model with 50 topics
topic_model = ct.Corex(n_hidden=45, words=words, max_iter=1200, verbose=False, seed=11)
topic_model.fit(doc_word, words=words);


In [ ]:
topics = topic_model.get_topics()
for n,topic in enumerate(topics):
    topic_words,_,_ = zip(*topic)
    print('{}: '.format(n) + ','.join(topic_words))

In [ ]:
topic_model.get_topics(topic=28, n_words=60)

In [95]:
anchor_words = [['worker', 'season', 'migrant', 'temporary']]
anchored_topic_modelt = ct.Corex(n_hidden=45, seed=10)
anchored_topic_modelt.fit(doc_word, words=words, anchors=anchor_words, anchor_strength=3)
for n in range(len(anchor_words)):
    topic_words,_,_ = zip(*anchored_topic_modelt.get_topics(topic=n))
    print('{} '.format(n) + ','.join(topic_words))


0 important,worker,country,including,national,canada,issue,cost,temporary,member


In [ ]:
anchored_topic_modelt.get_topics(topic=0, n_words=100)

mid covid

In [48]:
initialDF_midCovid= pd.read_csv('Hansard_mid_covid.csv', sep=',', encoding='utf-16-le')
initialDF_midCovid.drop(['Publication', 'First Name', 'Last Name','Constituency', 'Province', 'Date', 'Time', 'Page'], axis=1)
clearWords = stopwords.words('english')
new_Stopwords = ['in', 'the', 'said', 'like','must', 'many', 'also']
clearWords.extend(new_Stopwords)
lemma= WordNetLemmatizer()

initialDF_midCovid['Text'] = initialDF_midCovid['Text'].astype(str).str.lower()
initialDF_midCovid['Text'] = initialDF_midCovid['Text'].str.replace("|".join([r"^.*?speaker,", r"^.*?chair," ]), ' ', regex=True)
initialDF_midCovid['text_tokens'] = initialDF_midCovid['Text'].apply(word_tokenize)
initialDF_midCovid['text_tokens'] = initialDF_midCovid['text_tokens'].apply(lambda x: [item for item in x if item not in clearWords])
initialDF_midCovid['text_tokens'] = initialDF_midCovid['text_tokens'].apply(lambda x: [lemma.lemmatize(item) for item in x])
# do we love me or what? isn't this join thing amazing?
initialDF_midCovid['text_tokens'] = initialDF_midCovid['text_tokens'].apply(lambda x:' '.join([item for item in x if len(item)>3]))

In [49]:
vectorizer = CountVectorizer(stop_words='english', max_features=20000, binary=True)
doc_word = vectorizer.fit_transform(initialDF_midCovid['text_tokens'].astype(str))
doc_word = ss.csr_matrix(doc_word)
doc_word.shape # n_docs x m_words

# Get words that label the columns (needed to extract readable topics and make anchoring easier)
words = list(np.asarray(vectorizer.get_feature_names_out()))
not_digit_inds = [ind for ind,word in enumerate(words) if not word.isdigit()]
doc_word = doc_word[:,not_digit_inds]
words    = [word for ind,word in enumerate(words) if not word.isdigit()]


In [75]:
midcovid_unsup =  ct.Corex(n_hidden=60, words=words, max_iter=1200, verbose=False, seed=7)
midcovid_unsup.fit(doc_word, words=words);



In [ ]:
topics = midcovid_unsup.get_topics()
for n,topic in enumerate(topics):
    topic_words,_,_ = zip(*topic)
    print('{}: '.format(n) + ','.join(topic_words))

In [ ]:
midcovid_unsup.get_topics(topic=55, n_words=100)

In [87]:
anchor_words = [['workers', 'season', 'migrant']]
anchored_midCovidt= ct.Corex(n_hidden=45, seed=10)
anchored_midCovidt.fit(doc_word, words=words, anchors=anchor_words, anchor_strength=3)
for n in range(len(anchor_words)):
    topic_words,_,_ = zip(*anchored_midCovidt.get_topics(topic=n))
    print('{} '.format(n) + ','.join(topic_words))

0 migrant,workers,question,season,rising,proposes,floor,flexible,reminded,middle


In [ ]:
anchored_midCovidt.get_topics(topic =0, n_words=100)

In [100]:
from corextopic import vis_topic as vt
vt.vis_rep(topic_model, column_label=words, prefix='topic-model-example')

Print topics in text file


Hierarchical Modelling 

In [108]:
anchor_words = [['workers', 'season', 'migrant']]

new_mid_covid = ct.Corex(n_hidden=75)
new_mid_covid.fit(doc_word, words=words, anchors = anchor_words, anchor_strength=4)

ncm_layer2 = ct.Corex(n_hidden = 50)
ncm_layer2.fit(new_mid_covid.labels)    

ncm_layer3 = ct.Corex(n_hidden=20)
ncm_layer3.fit(ncm_layer2.labels)

ncm_layer4 = ct.Corex(n_hidden = 4)
ncm_layer4.fit(ncm_layer3.labels)


In [114]:
vt.vis_hierarchy([new_mid_covid, ncm_layer2, ncm_layer3, ncm_layer4],column_label=words, prefix='le' )

weight threshold is 0.210311 for graph with max of 100.000000 edges 
